In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing import List
import json

In [3]:
# 개발자 직무 30개
developer_roles = [
    (0, "Software Engineer"),
    (1, "Frontend Developer"),
    (2, "Backend Developer"),
    (3, "Full Stack Developer"),
    (4, "DevOps Engineer"),
    (5, "Cloud Engineer"),
    (6, "Data Engineer"),
    (7, "Machine Learning Engineer"),
    (8, "AI Engineer"),
    (9, "Embedded Software Engineer"),
    (10, "Mobile Developer"),
    (11, "iOS Developer"),
    (12, "Android Developer"),
    (13, "Game Developer"),
    (14, "Blockchain Developer"),
    (15, "Cybersecurity Engineer"),
    (16, "Systems Engineer"),
    (17, "Network Engineer"),
    (18, "Database Administrator"),
    (19, "QA Engineer"),
    (20, "Test Automation Engineer"),
    (21, "Site Reliability Engineer"),
    (22, "IoT Developer"),
    (23, "Big Data Engineer"),
    (24, "Business Intelligence Developer"),
    (25, "AR/VR Developer"),
    (26, "Firmware Engineer"),
    (27, "Solutions Architect"),
    (28, "Technical Support Engineer"),
    (29, "Software Architect")
]

In [ ]:
# 프롬프트
template = """
아래 예시 형식에 맞게 개발자를 희망하는 취업 준비생에게 필요한 프로젝트의 개요를 10개 작성해줘.

직무: {role}

<Example>
'title': '딥러닝 기반 영상-언어 융합 모델 개발', 'description': '영상과 텍스트 데이터를 융합하여 정보를 분석하고 예측하는 딥러닝 모델을 개발합니다. CNN과 RNN을 활용하여 다중 모달 데이터를 처리하고, 실험을 통해 모델의 성능을 검증합니다.', 'projectgoal': '영상-언어 융합 기술을 통해 다양한 응용 가능성을 탐색하고, 모델의 정확도를 높이는 것을 목표로 합니다.', 'techStacks': ['Python', 'TensorFlow', 'Keras', 'OpenCV'], 'qualifications': ['기계학습(딥러닝), 인공지능 전반에 대한 이론적, 기술적 이해', '영상-언어 융합 딥러닝 모델(CNNs, RNNs) 설계/개발 역량'], 'preferred_qualifications': ['기계학습(딥러닝), 이동체(차량, 비행체, 로봇), 인공지능 관련 최신 논문 이해 및 기법 재 구현 능력', '인공지능 관련 개발 지식/경험 보유 및 임베디드 경험'], 'userskillgaps': ['AI 신기술 평가 및 검증을 위한 실험 설계 및 실험 수행 역량', '확률 및 통계, 선형대수, 해석학 기반의 수학적 모델링에 대한 이해']
</Example>

<output_format>
{format_instructions}
</output_format>
"""

In [ ]:
# output 형식
os.chdir('/Users/no.2/Desktop/GitHub/cpplab-ai/project')
from models.project_info import Theme
class Themes(BaseModel):
    themes: List[Theme] = Field(..., description="생성된 프로젝트 개요의 리스트")

In [ ]:
# chain 생성
llm = ChatOpenAI(model="gpt-4o-mini")
prompt =  PromptTemplate(
    template=template,
    input_variables=['role']
)
parser = JsonOutputParser(pydantic_object=Themes)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())
chain = prompt | llm | parser

In [ ]:
# input data(프로젝트 개요) 생성
import asyncio
import json

# 직군 리스트, developer_roles 사용
# 비동기 호출을 위한 함수 정의
async def generate_project_data(role):
    input_data = {"role": role}
    result = await chain.ainvoke(input_data)
    print(f"Completed: {role}")
    return {role: result}

# 모든 직군에 대한 데이터를 비동기로 수집
async def main():
    tasks = [generate_project_data(role) for role in developer_roles]

    # 모든 작업을 병렬적으로 실행
    results = await asyncio.gather(*tasks)

    # 결과를 JSON 파일로 저장
    with open("/Users/no.2/Desktop/GitHub/cpplab-sllm/data/base/all_roles_input.json", "w", encoding="utf-8") as json_file:
        json.dump(results, json_file, ensure_ascii=False, indent=4)

    print("All roles processed and saved!")

# 비동기 실행
await main()

In [ ]:
# output data(프로젝트 가이드라인) 생성
import asyncio
import json

os.chdir('/Users/no.2/Desktop/GitHub/cpplab-ai/project/services')
from chain_generator import gendetails_chain

# 직군 리스트, developer_roles를 사용

MAX_RETRIES = 3

# 비동기 호출을 위한 함수 정의
async def generate_output_data(theme, retries=0):
    chain = gendetails_chain()

    try:
        result = await chain.ainvoke(
            input={
                "recommended_project": theme
            }
        )
        print(f"Completed: {theme['title']}")
        return {theme['title']: result}
    
    except Exception as e:
        print(f"Error processing {theme['title']}: {e}")
        if retries < MAX_RETRIES:
            print(f"Retrying {theme['title']} ({retries + 1}/{MAX_RETRIES})")
            return await generate_output_data(theme, retries=retries + 1)
        else:
            print(f"Failed to process {theme['title']} after {MAX_RETRIES} attempts")
            return {theme['title']: None}

# 모든 직군에 대한 데이터를 비동기로 수집
async def main(i, role):
    # JSON 파일 경로
    input_path = "/Users/no.2/Desktop/GitHub/cpplab-sllm/data/base/all_roles_input.json"
    output_path = "/Users/no.2/Desktop/GitHub/cpplab-sllm/data/base/all_roles_output.json"

    # 파일 읽기
    with open(input_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    tasks = [generate_output_data(theme) for theme in data[i][role]["themes"]]
    
    # 모든 작업을 병렬적으로 실행
    results = await asyncio.gather(*tasks)

    # 기존 파일 불러오기 (덮어쓰지 않도록)
    try:
        with open(output_path, "r", encoding="utf-8") as json_file:
            existing_results = json.load(json_file)
    except FileNotFoundError:
        existing_results = []

    # 새로운 결과 추가
    existing_results.extend(results)

    # 결과를 JSON 파일로 저장
    with open(output_path, "w", encoding="utf-8") as json_file:
        json.dump(existing_results, json_file, ensure_ascii=False, indent=4)
    print(f"Completed processing for role: {role}")

# 비동기 실행
for i, role in developer_roles:
    await main(i, role)
    print(f"Completed processing for role: {role}")

In [7]:
# 생성된 데이터 확인 274개
import json
import pandas as pd

# JSON 파일 경로
file_path = "/Users/no.2/Desktop/GitHub/cpplab-sllm/data/base/"

# 파일 읽기
with open(file_path + "all_roles_input.json", "r", encoding="utf-8") as file:
    inputs = json.load(file)
with open(file_path + "all_roles_output.json", "r", encoding="utf-8") as file:
    outputs = json.load(file)

# 데이터프레임 생성
data = []
j = 0
for i, role in developer_roles:
    for theme in inputs[i][role]['themes']:
        data.append({
            'role': role,
            'input': theme,
            'output': outputs[j][theme['title']]
        })
        j += 1

df = pd.DataFrame(data)
df['difficultyLevel'] = df['output'].apply(lambda x: x.get('difficultyLevel', None))
print(df.shape)
df.head()

(274, 4)


,role,input,output,difficultyLevel
0,Software Engineer,"{'title': '고급 웹 애플리케이션 개발', 'description': 'Re...","{'title': '고급 웹 애플리케이션 개발', 'description': 'Re...",중급
1,Software Engineer,"{'title': '모바일 앱 개발', 'description': 'Flutter를...","{'title': '모바일 앱 개발', 'description': 'Flutter를...",중급
2,Software Engineer,"{'title': '데이터 분석 및 시각화 프로젝트', 'description': ...","{'title': '데이터 분석 및 시각화 프로젝트', 'description': ...",중급
3,Software Engineer,"{'title': '웹 크롤러 및 데이터 수집', 'description': 'Be...","{'title': '웹 크롤러 및 데이터 수집', 'description': 'Be...",중급
4,Software Engineer,"{'title': '게임 개발 프로젝트', 'description': 'Unity를...","{'title': '게임 개발 프로젝트', 'description': 'Unity를...",중급
